# Ownlabelme2COCO  

In diesem Notebook wird der Code erklärt, der die Informationen in den einzelnen, durch die Software "Labelme" erzeugten, json-Dateien zu einer COCO-konformen json-Datei umstrukturiert.  
Inspiration für dieses Programm lieferte der [Blog-Post](https://www.dlology.com/blog/how-to-create-custom-coco-data-set-for-instance-segmentation/) vom GitHub-User Tony607. Sein Programm stellte sich jedoch als unkompatibel mit den erzeugten json-Dateien heraus, weswegen der folgende Code geschrieben wurde.  
Im Unterschied zu Tony607 wurde ein Objekt orientierter Ansatz gewählt.

In [1]:
import os
import json
import numpy as np

Die COCO-konformen Dateien für detectron2 setzen sich aus drei objektkategorien zusammen - Bilder, Kategorien und Annotationen. Für jede dieser Bezeichnungen wird zunächst eine Klasse erstellt.  
Alle Klassen setzen sich aus drei Funktionen zusammen. Der Konstruktor _init_ setzt die Klassenattribute, die _print_-Funktion gibt einen formatierten String mit allen Klassenattributen aus und die _convertToDictionary_ Funktion gibt ein Dictionary zurück, welches zum Export der Daten in die Zieldatei dient.  
Die Klassen unterscheiden sich in ihren Klassenattributen.

In [2]:
class Image:

    __doc__ = "The class, representing the image Data for the COCO Dataset"

    def __init__(self, id, name, height, width):
        self.id = id
        self.name = name
        self.height = height
        self.width = width

    def print(self):
        return '"height": {}\n,"width": {}\n,"id": {}\n,"file_name": {}\n'.format(
            self.height,
            self.width,
            self.id,
            self.name)

    def convertToDictionary(self):
        imagedict = {}
        imagedict["height"] = self.height
        imagedict["width"] = self.width
        imagedict["id"] = self.id
        imagedict["file_name"] = self.name

        return imagedict

In [3]:
class Category:

    __doc__ = "The class, representing the categorie Data for the COCO Dataset"

    def __init__(self, id, supercategory, name):
        self.id = id
        self.name = name
        self.supercategory = supercategory

    def print(self):
        return '"Supercategory: {}\n"id": {}\n"category": {}\n'.format(
            self.supercategory,
            self.id,
            self.name)
    
    def convertToDictionary(self):
        categorydict = {}
        categorydict["supercategory"] = self.supercategory
        categorydict["id"] = self.id
        categorydict["name"] = self.name

        return categorydict

In [4]:
class Polygon:
    
    __doc__ = "The class, representing the categorie Data for the COCO Dataset"

    def __init__(self, id, category_id, image_id, iscrowd, segmentation, bbox, area):
        self.id = id
        self.category_id = category_id
        self.image_id = image_id
        self.iscrowd = iscrowd
        self.segmentation = segmentation
        self.bbox = bbox
        self.area = area

    def print(self):
        return '"segmentation": {}\n"iscrowd": {}\n"area": {}\n"image_id": {}\n"bbox": {}\n"category_id": {}\n"id": {}\n'.format(
            self.segmentation,
            self.iscrowd,
            self.area,
            self.image_id,
            self.bbox,
            self.category_id,
            self.id)

    def convertToDictionary(self):
        polygondict = {}
        polygondict["segmentation"] = self.segmentation
        polygondict["iscrowd"] = self.iscrowd
        polygondict["area"] = self.area
        polygondict["image_id"] = self.image_id
        polygondict["bbox"] = self.bbox
        polygondict["category_id"] = self.category_id
        polygondict["id"] = self.id

        return polygondict

Die Funktion zur Errechnung der Poligonfläche wurde diesem [Stackoverflow Posting](https://stackoverflow.com/questions/24467972/calculate-area-of-polygon-given-x-y-coordinates) entnommen.

In [5]:
def PolyArea(x, y):
    return 0.5*np.abs(np.dot(x, np.roll(y, 1))-np.dot(y, np.roll(x, 1)))

In [6]:
WORK_DIR = "/home/julius/PowerFolders/Masterarbeit/"
os.chdir(WORK_DIR)

DATASET_PATH = "./1_Datensaetze/first_annotation_dataset/"

json_dump = {}
images = []
categories = []
polygons = []
label_list = {}
polygon_list = []

# get all json files in directory
json_list = sorted([f for f in os.listdir(DATASET_PATH) if f.endswith(".json")])

Das eigentliche Programm sammelt nach der Definition der Variablen alle .json Dateien im angegebenen Ordner in einer sortierten Liste.  
Danach wird über jede Datei in dieser Liste in mehreren for-Schleifen iteriert.  
In der ersten for-Schleife werden die nötigen Informationen für die _Image_ Klasse aus der Datei gezogen. In der zweiten Schleife dann die _Category_ Daten und abschließend die Koordinaten des Polygons. Innerhalb der zweiten for schleife werden die getrennten X und Y Koordinaten mit einander verzahnt, die _Bounding Box_ und der Flächeninhalt des Vielecks ermittelt.

In [7]:
for id_count, json_file in enumerate(json_list):
    with open(DATASET_PATH + json_file, "r") as content:
        data = json.load(content)
        
        image = Image(id_count, data["imagePath"].replace("../", ""), data["imageHeight"], data["imageWidth"])
        image_as_dict = image.convertToDictionary()
        images.append(image_as_dict)

        for shape_count, element in enumerate(data["shapes"]):
            if element["label"] not in label_list:
                category = Category((len(label_list)), None, element["label"])
                category_as_dict = category.convertToDictionary()
                categories.append(category_as_dict)
                label_list[element["label"]] = (len(label_list))

            x_coordinates = []
            y_coordinates = []

            # extract the polgon points
            for polygon in element["points"]:
                x_coordinates.append(polygon[0])
                y_coordinates.append(polygon[1])

            # transform into COCO format
            segmentation = []
            segmentation.append(list(sum(zip(x_coordinates, y_coordinates), ())))

            # get the values of the bbox
            smallest_x = int(min(x_coordinates))
            smallest_y = int(min(y_coordinates))
            biggest_x = int(max(x_coordinates))
            biggest_y = int(max(y_coordinates))

            bbox_height = biggest_y-smallest_y
            bbox_width = biggest_x-smallest_x

            bbox = [smallest_x, smallest_y, bbox_width, bbox_height]

            # get the area of the polygon
            polygon_area = PolyArea(x_coordinates, y_coordinates)

            # create polygon instance and add it to the list
            polygon = Polygon(len(polygon_list), label_list[element["label"]], id_count, 0, segmentation, bbox, polygon_area)
            polygon_as_dict = polygon.convertToDictionary()
            polygons.append(polygon_as_dict)
            polygon_list.append(shape_count)

Nach dem Auslesen und Sortieren der Daten werden diese in das Dictionary eingefügt, das auch die json Struktur der endgültigen Datei wiederspiegeln soll. Dieses wird am Ende in ein entsprechendes Unterverzeichnis im Datensatz eingefügt.

In [8]:
json_dump["images"] = images
json_dump["categories"] = categories
json_dump["annotations"] = polygons

output_path = DATASET_PATH + "COCO_json"
if not os.path.isdir(output_path):
    os.mkdir(output_path)

with open(output_path + "/output.json", "w") as output_file:
    json.dump(json_dump, output_file, indent=4)